<a id='menu'></a>

# Segmenting and Clustering Neighborhoods in Toronto
1. <a href="#menu1">__Part 1__</a> Scrapping neighborhood
2. <a href="#menu2">__Part 2__</a> Locating neighborhood
2. <a href="#menu3">__Part 3__</a> Clustering neighborhood

Importing libraries

In [4]:
import pandas as pd # library for data analysis
import requests
import folium
import os
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup #for scraping
from sklearn.cluster import KMeans

%matplotlib inline
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


<a id='menu1'></a>
## Part 1 Scrapping neighborhood

We are going to scraping the neighborhood data from Wikipedia. 

In [5]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(wiki)

In [7]:
soup = BeautifulSoup(r.content)
trs = soup.find('table','wikitable sortable').find_all('tr')
columns = ['postcode', 'borough', 'neighborhood']
df_postcode = pd.DataFrame(columns=columns)
for tr in trs:
    tds = tr.find_all('td')
    if len(tds) == 0: continue
    postcode = tds[0].get_text().strip()
    borough = tds[1].get_text().strip()
    neighborhood = tds[2].get_text().strip()
    if borough != 'Not assigned':
#         print("{} - {} - {}".format(postcode, borough, neighborhood))
        df_postcode = df_postcode.append({'postcode': postcode, 'borough': borough, 'neighborhood':neighborhood}, ignore_index=True)

print("Postcode count: {}".format(df_postcode.shape[0]))
df_postcode.head()

Postcode count: 211


,postcode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Data Preparation

Find ___"Not assigned"___ neighborhood and replace it with borough value

In [8]:
df_postcode[df_postcode.neighborhood=='Not assigned']

,postcode,borough,neighborhood
6,M7A,Queen's Park,Not assigned


In [9]:
df_postcode['neighborhood'] = df_postcode.apply(lambda x: x.borough if x.neighborhood == 'Not assigned' else x.neighborhood, axis=1)

In [10]:
df_postcode[df_postcode.neighborhood=='Not assigned']

,postcode,borough,neighborhood


Now there is no neighborhood with ___"Not Assigned"___ value

Next step is __grouping neighborhood__ with the same postcode into one row

In [12]:
df_postcode_grouped = df_postcode.groupby(['postcode','borough'], as_index=False).count()

df_postcode_clean = pd.DataFrame(columns=columns)
for index, row in df_postcode_grouped.iterrows():
    neighborhood = df_postcode[df_postcode.postcode==row[0]].neighborhood.str.cat(sep=',')
    df_postcode_clean = df_postcode_clean.append({'postcode':row[0], 'borough':row[1], 'neighborhood':neighborhood}, ignore_index=True)
    
df_postcode_clean.head()

,postcode,borough,neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<a href="#menu">_Back to top_</a>
<a id='menu2'></a>
## Part 2 Locating neighborhood

Find __coordinate__ of each postcode by reading from https://cocl.us/Geospatial_data

In [28]:
coordinate_csv = 'data/coordinate.csv'
# read from file downloaded before if available
if os.path.exists(coordinate_csv):
    df_coordinate = pd.read_csv('data/coordinate.csv')
else:
    url = 'https://cocl.us/Geospatial_data'
    df_coordinate = pd.read_csv(url)    

Postcode' coordinates dataframe:

In [14]:
df_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


__Join__ with df_postcode_clean dataframe

In [15]:
df_postcode_coordinate = df_postcode_clean.join(df_coordinate.set_index('Postal Code'), on='postcode')
df_postcode_coordinate.head()

,postcode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
df_postcode_coordinate.shape

(103, 5)

__All__ postcode now has coordinates

<a id='menu3'></a>

<a href="#menu">_Back to top_</a>
## Part 3 Clustering neighborhood

In [34]:
map_city = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

for lat, long, postcode, borough, neighborhood in zip(df_postcode_coordinate.Latitude, df_postcode_coordinate.Longitude, df_postcode_coordinate.postcode, df_postcode_coordinate.borough, df_postcode_coordinate.neighborhood):
        folium.CircleMarker(
            [lat, long],
            radius=5,
            color='yellow',
            fill=True,
            popup="<b>{}</b>\n{}:\n{}".format(postcode, borough, neighborhood),
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(map_city)

map_city

In [35]:
CLIENT_ID = 'WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG' # your Foursquare ID
CLIENT_SECRET = 'XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG
CLIENT_SECRET:XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY


#### Let's explore the first neighborhood in our dataframe.

In [36]:
neighborhood_latitude, neighborhood_longitude = df_postcode_coordinate.iloc[0].Latitude, df_postcode_coordinate.iloc[0].Longitude
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v=20180605&ll={},{}&radius=500&limit=100".format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG&client_secret=XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY&v=20180605&ll=43.8066863,-79.19435340000003&radius=500&limit=100


In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd16a859fb6b7571a10235b'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.18812958073043},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'di

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## 2. Explore Neighborhoods in City of Toronto

Again borrowing funtion from the course

In [91]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [92]:
district_venues = getNearbyVenues(names=df_postcode_coordinate['neighborhood'], latitudes=df_postcode_coordinate['Latitude'],longitudes=df_postcode_coordinate['Longitude'],radius=500)

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [93]:
district_venues.shape[0]

2243

In [94]:
district_venues.head()

,Neighborhoods,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


Let's check how many venues were returned for each neighborhood

In [95]:
district_venues.groupby('Neighborhoods', as_index=False).count()

,Neighborhoods,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide,King,Richmond",100,100,100,100,100,100
1,Agincourt,4,4,4,4,4,4
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",3,3,3,3,3,3
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",9,9,9,9,9,9
4,"Alderwood,Long Branch",10,10,10,10,10,10
5,"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
6,Bayview Village,4,4,4,4,4,4
7,"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
8,Berczy Park,57,57,57,57,57,57
9,"Birch Cliff,Cliffside West",4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [96]:
print('There are {} uniques categories.'.format(len(district_venues['Venue Category'].unique())))

There are 274 uniques categories.


<a id='menu3'></a>

## 3. Analyze Each Neighborhood

In [98]:
# one hot encoding
district_onehot = pd.get_dummies(district_venues[['Venue Category']], prefix="", prefix_sep="")

#Add Neighborhood name
district_onehot = pd.concat([district_venues[['Neighborhoods']], district_onehot], axis=1)

district_onehot.head(10)

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [99]:
district_onehot.shape

(2243, 275)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [101]:
district_grouped = district_onehot.groupby('Neighborhoods').mean().reset_index()
district_grouped.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


#### Let's confirm the new size

In [102]:
district_grouped.shape

(100, 275)

#### Let's print each neighborhood along with the top 5 most common venues

In [104]:
num_top_venues = 5

for hood in district_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = district_grouped[district_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True)[:5])
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3        Skating Rink  0.25
4  Miscellaneous Shop  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground  0.33
1         Coffee Shop  0.33
2                Park  0.33
3  Miscellaneous Shop  0.00
4       Movie Theater  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1              Pharmacy  0.11
2  Fast Food Restaurant  0.11
3   Fried Chicken Joint  0.11
4           Coffee Shop  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place   0.2
1 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = district_grouped['Neighborhoods']

for ind in np.arange(district_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(district_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Pizza Place,Dumpling Restaurant,Drugstore
4,"Alderwood,Long Branch",Pizza Place,Pool,Pub,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Department Store


<a id='menu4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [108]:
district_grouped.shape

(100, 275)

In [109]:
# set number of clusters
kclusters = 5

district_grouped_clustering = district_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(district_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [111]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,2,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
2,1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Pizza Place,Dumpling Restaurant,Drugstore
4,2,"Alderwood,Long Branch",Pizza Place,Pool,Pub,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Department Store


In [121]:
district_merged = neighborhoods_venues_sorted.join(df_postcode_coordinate.set_index('neighborhood'), on='Neighborhoods')

In [122]:
cols = district_merged.columns.tolist()
cols = cols[-4:]+[cols[1]]+[cols[0]]+cols[2:-4]
district_merged = district_merged[cols]

district_merged.head() # check the last columns!

,postcode,borough,Latitude,Longitude,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,43.650571,-79.384568,"Adelaide,King,Richmond",2,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,M1S,Scarborough,43.794200,-79.262029,Agincourt,2,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
2,M1V,Scarborough,43.815252,-79.284577,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1,Park,Playground,Coffee Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,M9V,Etobicoke,43.739416,-79.588437,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",2,Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Pizza Place,Dumpling Restaurant,Drugstore
4,M8W,Etobicoke,43.602414,-79.543484,"Alderwood,Long Branch",2,Pizza Place,Pool,Pub,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Department Store


Finally, let's visualize the resulting clusters

In [124]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(district_merged['Latitude'], district_merged['Longitude'], district_merged['Neighborhoods'], district_merged['Cluster Labels'].astype(np.int64)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='menu5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [130]:
district_merged.loc[district_merged['Cluster Labels'] == 0, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

,postcode,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M7R,Canada Post Gateway Processing Centre,0,Hotel,Coffee Shop,American Restaurant,Gym / Fitness Center,Sandwich Place,Mediterranean Restaurant,Burrito Place,Fried Chicken Joint,Diner,Discount Store
96,M1G,Woburn,0,Coffee Shop,Korean Restaurant,Insurance Office,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 2

In [129]:
district_merged.loc[district_merged['Cluster Labels'] == 1, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

,postcode,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1V,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1,Park,Playground,Coffee Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
13,M3K,"CFB Toronto,Downsview East",1,Park,Airport,Snack Place,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
16,M6E,Caledonia-Fairbanks,1,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
41,M4J,East Toronto,1,Park,Convenience Store,Metro Station,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
74,M4W,Rosedale,1,Park,Playground,Trail,Yoga Studio,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
80,M2L,"Silver Hills,York Mills",1,Cafeteria,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
89,M8X,"The Kingsway,Montgomery Road,Old Mill North",1,Park,River,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
93,M9N,Weston,1,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
99,M2P,York Mills West,1,Bank,Park,Electronics Store,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 3

In [131]:
district_merged.loc[district_merged['Cluster Labels'] == 2, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

,postcode,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,"Adelaide,King,Richmond",2,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,M1S,Agincourt,2,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
3,M9V,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",2,Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Pizza Place,Dumpling Restaurant,Drugstore
4,M8W,"Alderwood,Long Branch",2,Pizza Place,Pool,Pub,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Department Store
5,M3H,"Bathurst Manor,Downsview North,Wilson Heights",2,Coffee Shop,Sandwich Place,Supermarket,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Fast Food Restaurant
6,M2K,Bayview Village,2,Bank,Café,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,M5M,"Bedford Park,Lawrence Manor East",2,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Greek Restaurant,Thai Restaurant,Liquor Store,Sandwich Place,Juice Bar,Butcher,Restaurant
8,M5E,Berczy Park,2,Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café
9,M1N,"Birch Cliff,Cliffside West",2,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,M9C,"Bloordale Gardens,Eringate,Markland Wood,Old B...",2,Pharmacy,Liquor Store,Café,Convenience Store,Beer Store,Pizza Place,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 4

In [132]:
district_merged.loc[district_merged['Cluster Labels'] == 3, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

,postcode,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,M1C,"Highland Creek,Rouge Hill,Port Union",3,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dessert Shop


#### Cluster 5

In [133]:
district_merged.loc[district_merged['Cluster Labels'] == 4, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

,postcode,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,M9M,"Emery,Humberlea",4,Construction & Landscaping,Baseball Field,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
56,M8Y,"Humber Bay,King's Mill Park,Kingsway Park Sout...",4,Baseball Field,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


# Thank You !!
<a href="#menu">_Back to Top_</a>